In [ ]:

import os
import geopandas as gpd

import matplotlib.pyplot as plt
import geopandas as gpd
import basic
import contextily as ctx
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import os
import matplotlib.pyplot as plt
import pathlib
import conda_scripts.plot_help as ph
from scipy.spatial.distance import pdist, squareform
from matplotlib import ticker

import write_inflows
import flopy.utils.binaryfile as bf

import warnings

In [ ]:
info, swr_info, sfr_info, riv_keys_info = basic.load_params('June2016')

datestart = info['start_date']

name = info['name']

out_folder = basic.out_folder('June2016')


print(datestart)
print(out_folder)

In [ ]:
import conda_scripts.arich_functions as af

In [ ]:
basic

In [ ]:
grid = gpd.read_file("GIS/grid.shp")


In [ ]:
ml = basic.load_model()


In [ ]:
routing = gpd.read_file('GIS/nhd_hr_demo_sfr_routing.shp')

In [ ]:
# bottom = 765


In [ ]:
# kper = None

In [ ]:
# def load_params():
    
#     import json

#     with open("model_params.txt") as json_data_file:
#         data = json.load(json_data_file)
    
#     info = data['calibration']
#     swr = data['calibration']['parameters']['SWR']
#     sfr = data['calibration']['parameters']['SFR']
#     riv_keys = data['riv_keys']
    
#     return info, swr, sfr, riv_keys



info, swr_info,sfr_info, riv_keys = basic.load_params(run_name = 'June2013')
riv_keys

In [ ]:
swr_info

In [ ]:
IGEOTYPE = swr_info["IGEOTYPE"] 
IGCNDOP = swr_info["IGCNDOP" ]
GMANNING = swr_info["GMANNING" ]   
LEAKANCE = swr_info["LEAKANCE"]
getextd = swr_info["getextd"] 
bottom = swr_info['dam_swr_rno']
reach_min = swr_info["reach_main"]
IROUTETYPE = swr_info["IROUTETYPE"]
use_average_reach_len = swr_info["use_average_reach_len"]
use_thalweg = swr_info['use_thalweg']
kper = None if swr_info["kper"].lower()=='none' else swr_info["kper"]
SWR_processor = swr_info["SWR_processor"]
rain = swr_info['rain']
evap = swr_info['evap']

kper

In [ ]:

datestart = '1/1/2012'
enddate= '1/1/2022'

numdays = (pd.to_datetime(enddate) - pd.to_datetime(datestart)).days
rr, dry, mw, total, stg = write_inflows.run('June2012', m = ml, minvalue=0, max_value = 100, numdays = numdays, datestart = '1/1/2012', cleandamdata = False)

In [ ]:
stg = stg.drop(columns = ['FillValue'])

In [ ]:
fig, ax = plt.subplots(figsize= (6,6))


def line(flow):
    
    minflow = 38
    yint = 28
    slope1 = .0023
    slope2 = .0005
    slope3 = .0002
    cutoff = 7000
    
    cutoff2 = 45000
    
    if flow < 500:
        return 28.5
    elif flow<cutoff:
        return yint+slope1*flow
    elif flow < cutoff2:
        return yint+13+slope2*flow
    else:
        return yint+26.5+slope3*flow
    
scatter = ax.scatter(stg.rrtotal, stg.Original_Value, c = stg.index.year, facecolor = 'None', marker = 'o', alpha = .9,cmap  = plt.get_cmap('tab20'))
ax.grid(True)
x = np.arange(1, 75000,500)
y = [line(xi) for xi in x]

l = ax.plot(x, y, 'r', marker = '.', label = 'Rating Curve')

ax.set_xlabel('Russian River Discharge (Healdsburg + Dry Creek Flows) cfs')
ax.set_ylabel('RDS Observed Stage (ft)')
ax.set_title('Stage Discharge Relationship for\nEstimating Overland Flows Along Russian River')

legend1 = ax.legend(*scatter.legend_elements(),
                    title="Year", loc = 'lower right', bbox_to_anchor = (1,0))
ax.add_artist(legend1, )

# plt.savefig('Overland_Flow/ovr_relationship.png', dpi = 300)
# plt.savefig('versions/website_info/ovr_relationship.png', dpi = 300)

In [ ]:
stg.loc[:,'stage_est'] = stg.rrtotal.apply(line)

# stg = stg.loc['2019',:]

In [ ]:
stg.loc[:,'Original_Value':].plot(subplots = True)


In [ ]:
stg.plot(subplots = True)

In [ ]:

plt.figure( figsize = (6,6), dpi = 300)
ax = stg.loc[:,'stage_est'].plot( label = 'Estimated Stage')
stg.loc[:,'Original_Value'].plot( label = 'Actual Stage', ls = '--')
stg.query('stage_est>40').loc[:,'stage_est'].plot(ax = ax, marker = '.', ls = 'None', label = 'Stage Exceedance Events')
ax.set_ylabel('ft')
ax.axhline(40, c = 'k')
ax.text(1, 40,   'Cutoff', ha = 'left', va = 'center', transform = ax.get_yaxis_transform())
ax.axhline(28, c = 'brown')
ax.text(1, 28,   'Ground Surface', ha = 'left', va = 'center', transform = ax.get_yaxis_transform())

date = stg.stage_est.idxmax()+pd.to_timedelta(10, 'D')
maxvalue = stg.stage_est.max()
ax.set_ylim([25, 70])

ax.annotate('Offset Value', xy=(date, 28),
            ha = 'center',
             xycoords='data',
             xytext=(date, maxvalue),
             textcoords='data',
             arrowprops=dict(arrowstyle= '|-|',
                             color='blue',
                             lw=3,
                             ls='--')
           )

ax.set_title("RDS Estimated Stage")

ax.legend()

# plt.savefig('Overland_Flow/stage_exceedance.png', dpi = 300, bbox_inches = 'tight')
# plt.savefig('versions/website_info/stage_exceedance.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
    varsall = dict(mult_shear = .005,
    mult_sed = .05,
    sedlimit = 5,
    power = 1/3,
    sed_depth_rate = .025)

In [ ]:


def get_fac(par):

    
    # fac = np.array([1/20, 1/10, 1/2, 1., 2, 10, 20])
    fac = np.array([ 1/4, 1/2, 1., 2, 4])
    
    return fac

def sed(stages, 
    mult_shear = None,
    mult_sed = None,
    sedlimit = None,
    sedlast = None,
    power = None,
    sed_depth_rate = None):
      
    mult_shear = varsall['mult_shear'] if mult_shear is None else mult_shear
    mult_sed = varsall['mult_sed'] if mult_sed is None else mult_sed
    sedlimit = varsall['sedlimit'] if sedlimit is None else sedlimit
    # sedlast = varsall['sedlast'] if sedlast is None else sedlast
    power = varsall['power'] if power is None else power
    sed_depth_rate = varsall['sed_depth_rate'] if sed_depth_rate is None else sed_depth_rate
    
    st = stages.copy()
    
    sedlast = 0.

    st.loc[:,'Shear'] = mult_shear*((st.loc[:,'rrtotal'] )**(power))
    st.loc[:,'erode'] = np.nan
    
    st.loc[:,'Sedrate'] = np.nan
    
    for ind, row in st.iterrows():

        erode = st.loc[ind, 'Shear']
        if row.loc['Value']>10:
            erode = 0
        
        st.loc[ind, 'erode'] = erode
        
        sedrate =  sed_depth_rate -  erode
        sedlast = sedlast + sedrate

        if sedlast<0:
            sedlast =0
        elif sedlast>sedlimit:
            sedlast = sedlimit
        
        st.loc[ind, 'Sedrate'] = sedrate
        st.loc[ind, 'sed_depth'] = sedlast 
        
        
        if np.isnan(sedlast) or np.isnan(sedrate):
            print(sedlast, sedrate)
            raise ValueError('sedlast or sedrate are nans')
        
    return st


In [ ]:

st_q = sed(stg)
st_q.drop(columns = [ 'Original_Value','Value']).plot(subplots = True)

In [ ]:
def plot_par(var, stages, base_only = False):
    
    
    fig, ax = plt.subplots(figsize = (7,7))

    if base_only:
        
        stages = sed(stages)
        
        ax.plot(stages.index, stages.loc[:,'sed_depth'], label =var, c = 'r' )
        
    else:
        par_range = get_fac(var)

        for seddepth in par_range:
            stages = sed(stages, **{var:seddepth * varsall[var]})

            ax.plot(stages.index, stages.loc[:,'sed_depth'], label =f'{seddepth * varsall[var]:.3G} ({seddepth:.3G}x)' )

    
    ax.legend(bbox_to_anchor = (1,1), loc = 'upper left')
    ax.set_ylabel('sediment depth (units unknown)', color = 'r')
    ax.set_title(f"{var} Parameter Sensitivity")
    ax.grid(True)
    # plt.savefig(os.path.join('Sedimentation', var+'.png'), bbox_inches = 'tight', dpi = 250)
    
    return ax
    


In [ ]:
[plot_par(vari, stg) for vari in varsall.keys()]

In [ ]:
import textwrap
ax = plot_par('sed_depth_rate', stg, True)
ax.legend(bbox_to_anchor = (1,0), loc = 'upper left')
ax2 = ax.twinx()

att = dict(a=(5,'c',1), b=(5,'g',2), c=(2,'lightgrey',3), d=(2,'grey',4),e=(2,'darkgrey',5),f=(6,'b',6))

for _, mm in  att.items():
    caiss = mm[2]
    caisson6 = dfall.query(f"station_no=='Caisson{caiss}'").dropna(0,subset = 'Observed')
    caisson6 = caisson6.loc[(caisson6.index.month>5) & (caisson6.index.month<11)]
    caisson6.loc[:,'Resid'] = -(caisson6.loc[:,'Observed'] -  caisson6.loc[:,'Simulated'])
    caisson6.loc[:,'Resid'] = caisson6.loc[:,'Resid'] - caisson6.loc[:,'Resid'].mean()
    cam = caisson6.resample('1M').median()
    # ax2.plot(cam.resample('1Q').median().dropna().index, cam.resample('1Q').median().dropna().Resid, lw = .5,  ls = '-', 
    #          marker = None, markersize = mm[0], color = mm[1], label = 'nolabel')
    ax2.plot(cam.index, cam.Resid, lw = 2, marker = 'o', ls = 'None', markersize = mm[0], color = mm[1], label = f"Caisson {caiss}")

ax.set_title('Esimated Sediment Depth,\nPotential Clogging as Indicated by Model Residuals')
ax2.set_ylim([-30, 20])
ax2.set_ylabel('Residual (Normalized - feet)', color = 'k')
text = '\n'.join(textwrap.wrap('Residuals are the difference between the observed and simulated groundwater levels. Greater differences are interpreted to indicate thicker clogging layer', 20))
props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
ax2.text(1.1, 0,text, transform = ax.transAxes,  bbox=props  )
ax2.legend(bbox_to_anchor = (1.1,1), loc = 'upper left')

plt.savefig(os.path.join('Sedimentation', 'model_clogging.png'), bbox_inches = 'tight', dpi = 250)

In [ ]:
import fiona
import geopandas as gpd

In [ ]:
ax = cam.loc[:,['Simulated','Observed']].plot()

ax2 = ax.twinx()

cam.loc[:,'Resid'].plot(ax = ax2, c = 'r')

In [ ]:
dfall = pd.DataFrame()

for year in range(2012, 2021):
    file = os.path.join('versions', f'June{year}', 'hydrographs', 'all_meas.csv')
    df = pd.read_csv(file, index_col=[0], parse_dates= True)
    
    dfall = dfall.append(df)

In [ ]:
grid = gpd.read_file("GIS/grid.shp")

# need to update model top for streambed locations
grid.loc[:,'model_top'] = ml.dis.top.array[grid.loc[:,'row']-1, grid.loc[:,'column']-1]
grid.loc[:,'ibound'] = ml.bas6.ibound.array[0][grid.loc[:,'row']-1, grid.loc[:,'column']-1]


m = grid.query("ibound==1").explore('model_top',vmax = 75, vmin = 30)
sfr.set_geometry(sfr.geometry.centroid).explore( m=m)

In [ ]:

grid_simp = grid.query("ibound==1").loc[:,['model_top','node','geometry']].rename(columns ={'node':'node_grid'})
sfr_simp = sfr.set_geometry(sfr.geometry.centroid).loc[:,[ 'geometry', 'node', 'iseg', 'ireach']].rename(columns ={'node':'node_sfr'}).drop_duplicates('node_sfr')

gridnear = gpd.sjoin_nearest(grid_simp,
                            sfr_simp,
                            how = 'left')

print(f"shape {gridnear.shape[0]}")
gridnear = gridnear.drop_duplicates('node_grid')
print(f"shape {gridnear.shape[0]}")

In [ ]:
dfall = pd.DataFrame()
for date, row in stg.query('stage_est>40').iterrows():
    s = sfr.copy().drop_duplicates('node').loc[:,['model_top', 'node']].rename(columns = {'node':'node_sfr'})
    s.loc[:,'Date'] = date
    s.loc[:,'offset'] = row['stage_est']-28
    s.loc[:,'WSE'] = s.loc[:,'model_top'] + s.loc[:,'offset']
    s = s.drop(columns = 'model_top')
    dfall = dfall.append(s)
    


In [ ]:
floods = pd.merge(dfall, gridnear, on = 'node_sfr')

floods = floods.query("WSE>model_top")

fig = plt.figure(figsize = (6,6), dpi =300)
ax = floods.groupby('Date').count().loc[:,'WSE'].plot(marker = 'o', ls = 'None')
ax.grid(True)
ax.set_title('Number of Overland flow events')
ax.set_ylabel('Number of cells with overland flow events')
ax.set_xlabel('')
plt.savefig('Overland_Flow/stage_exceedance_count.png', dpi = 300, bbox_inches = 'tight')
plt.savefig('versions/website_info/stage_exceedance_count.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
import conda_scripts.utils.folium_maps as fm

In [ ]:
floods_sum = pd.merge(floods.drop(columns = 'geometry').groupby('node_grid').count().reset_index().loc[:,['node_grid','WSE']], 
                      gridnear.drop_duplicates('node_grid').loc[:,['node_grid', 'geometry','model_top', 'node_sfr']], on = 'node_grid', how = 'right')
# floods_sum = floods_sum.fillna(0)
floods_sum = floods_sum.rename(columns = {'WSE': 'Number of flood days'})
floods_sum = gpd.GeoDataFrame(floods_sum, geometry = 'geometry', crs = 2226)
m = floods_sum.explore('Number of flood days', tooltip  = ['Number of flood days', 'model_top', 'node_sfr'], name = 'Number of flood days')

# gridnear.explore('model_top',tooltip  = [ 'model_top'], m = m, name = 'gridnerar', vmin = 20, vmax = 75)
# sfr.explore( 'model_top', m=m, color= 'r', name = 'sfr', vmin = 20, vmax = 75)
sfr.set_geometry(sfr.geometry.centroid).explore( m=m, color= 'r', name = 'SFR Cells')
fm.add_layers(m)
fm.add_title('Number of Overland Flow Events per Cell, from 2012 to 2021',m)
m.save('versions/website_info/overland_flow_count.html')
m.save('Overland_Flow//overland_flow_count.html')

In [ ]:
floods.sort_values(['Date', 'node_grid']).loc[:,['Date','node_grid','WSE', 'model_top']].to_csv('Overland_Flow/overland_flow_ts.csv',
                                                                                                float_format="%.0f",index = False)

In [ ]:
def read_overland(m):
    ovr = pd.read_csv('Overland_Flow/overland_flow_ts.csv', index_col = [0],parse_dates=True )
    ovr.loc[:, ['k','i', 'j']] = m.dis.get_lrc(list(ovr.loc[:,'node_grid'].values))
    
    return ovr

ovr = read_overland(ml)

In [ ]:
1000*86400/43560

In [ ]:
def plot_ovr(ovr, datestart, folder, recharge_rate, numdays = 365):
    
    q = ovr.groupby(ovr.index).count().loc[:,['WSE']] * 200*200*recharge_rate/43560
    end_date = (pd.to_datetime(datestart ) + pd.to_timedelta(numdays+5, 'D')).strftime('%m/%d/%Y')
    q = q.resample('1D').sum()
    qm = q.max()
    q = q.loc[datestart:end_date,:].rename(columns = {'WSE':"Overland Recharge"})
    plt.figure( figsize = (6,6), dpi =300)
    ax = q.plot(drawstyle="steps-post", linewidth=2, ylabel = 'recharge (acre-feet)', c = 'b')
    ax.set_ylim([0, qm.values[0]]); 
    ax.grid(True); 
    ax.yaxis.get_label().set_color('b')
    # ax.text(1,1, f'Recharge rate = {recharge_rate}ft.', transform = ax.transAxes, ha = 'right', va = 'bottom')
    ax.legend().remove()
    ax.set_title(f'Daily Recharge from Overland Flow.\nRecharge rate = {recharge_rate}ft/d')
    
    ax2 = ax.twinx()
    
    p2 = q.cumsum().plot(ax = ax2, label = 'Cumulative', c = 'r', ylabel = 'cumulative recharge (acre-feet)')
    ax.get_yaxis().set_major_formatter(
            ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    ax2.get_yaxis().set_major_formatter(
        ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    ax2.yaxis.get_label().set_color('r')
    
    plt.savefig(os.path.join(folder, 'ovr_total.png'), bbox_inches = 'tight', dpi = 250, figsize = (6,6))
    
    return ax

for year in range(2012, 2021):
    z = plot_ovr(ovr, f'1/1/{year}', f'versions/June{year}', 0.5)

In [ ]:
gridnear.explore('node_sfr')